# Generating Dashboards

In [2]:
from IPython.display import display, HTML
import torch
from datasets import load_dataset
import webbrowser
import os
import sys
from transformer_lens import utils, HookedTransformer
from tqdm.notebook import tqdm
from datasets.arrow_dataset import Dataset
from huggingface_hub import hf_hub_download
from sae_vis.model_fns import AutoEncoder, AutoEncoderConfig
from sae_vis.data_fetching_fns import get_feature_data, get_prompt_data
from sae_vis.data_storing_fns import FeatureVisParams, FeatureData, MultiFeatureData, MultiPromptData, save_json, load_json
from sae_vis.utils_fns import create_vocab_dict, get_device

device = get_device()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.set_grad_enabled(False);

ModuleNotFoundError: No module named 'torch'

In [11]:
from sae_analysis.dashboard_runner import DashboardRunner


layer = 8
REPO_ID = "jbloom/GPT2-Small-SAEs"
FILENAME = f"final_sparse_autoencoder_gpt2-small_blocks.{layer}.hook_resid_pre_24576.pt"
path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)

# Make sure Joseph's lib is in the path, or else the load will fail
if os.getcwd() + "/mats_sae_training" not in sys.path:
    sys.path.append(os.getcwd() + "/mats_sae_training")

obj = torch.load(path, mmap="cpu")
state_dict = obj["state_dict"]
assert set(state_dict.keys()) == {'W_enc', 'b_enc', 'W_dec', 'b_dec'}

cfg = AutoEncoderConfig(
    d_in = obj["cfg"].d_in,
    dict_mult = obj["cfg"].expansion_factor,
)
gpt2_sae = AutoEncoder(cfg)
gpt2_sae.load_state_dict(state_dict)


runner = DashboardRunner(
    sae_path=path,
    dashboard_parent_folder="../feature_dashboards",
    init_session=True,
    n_batches_to_sample_from=2**12,
    n_prompts_to_select=4096 * 6,
    n_features_at_a_time=128,
    max_batch_size=256,
    buffer_tokens=8,
    use_wandb=False,
    continue_existing_dashboard=True,
)
runner.run()


Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda


/home/paperspace/miniconda3/envs/mats_sae_training/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Run name: 24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.004096
Total training steps: 73242
Total wandb updates: 732
n_tokens_per_feature_sampling_window (millions): 524.288
n_tokens_per_dead_feature_window (millions): 2621.44
Using Ghost Grads.
We will reset the sparsity calculation 73 times.
Number tokens in sparsity calculation window: 4.10e+06
Run name: 24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.004096
Total training steps: 73242
Total wandb updates: 732
n_tokens_per_feature_sampling_window (millions): 524.288
n_tokens_per_dead_feature_window (millions): 2621.44
Using Ghost Grads.
We will reset the sparsity calculation 73 times.
Number tokens in sparsity calculation window: 4.10e+06
File 0 does not exist
features left to do: 24576
id_to_start_from: 0
number of batches to do: 192
Hook Point Layer: 8
Hook Point: 

100%|██████████| 4096/4096 [01:35<00:00, 42.69it/s]


Time to get tokens: 95.98009657859802


...

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]


Forward passes to gather data:   0%|          | 0/384 [00:00<?, ?it/s]

Getting sequence data:   0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.54 GiB. GPU 0 has a total capacity of 79.33 GiB of which 838.62 MiB is free. Process 13836 has 70.38 GiB memory in use. Including non-PyTorch memory, this process has 8.11 GiB memory in use. Of the allocated memory 6.81 GiB is allocated by PyTorch, and 828.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [4]:
gpt2_sae.hook_point

AttributeError: 'AutoEncoder' object has no attribute 'hook_point'

In [10]:
import re
re.match(r'blocks\.\d\.', "blocks.10.hook_resid_pre_24576.pt")